In [33]:
import numpy as np
import math

def gauss_elimination(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    n = len(b)

    for k in range(0, n - 1):
        for i in range(k + 1, n):
            l = a[i, k] / a[k, k]
            a[i, k + 1:n] = a[i, k + 1:n] - l * a[k, k + 1:n]
            b[i] = b[i] - l * b[k]

    for k in range(n - 1, -1, -1):
        b[k] = (b[k] - np.dot(a[k, k + 1:n], b[k + 1:n])) / a[k, k]

    return b


def lu_decomposition(a: np.ndarray) -> np.ndarray:
    n = len(a)

    for k in range(0, n - 1):
        for i in range(k + 1, n):
            l = a[i, k] / a[k, k]
            a[i, k + 1:n] = a[i, k + 1:n] - l * a[k, k + 1:n]
            a[i, k] = l

    return a


def lu_solve(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    n = len(a)

    for k in range(1, n):
        b[k] = b[k] - np.dot(a[k, 0:k], b[0:k])

    b[n - 1] = b[n - 1] / a[n - 1, n - 1]

    for k in range(n - 2, -1, -1):
        b[k] = (b[k] - np.dot(a[k, k + 1:n], b[k + 1:n])) / a[k, k]

    return b


def choleski_decomposition(a: np.ndarray) -> np.ndarray:
    n = len(a)

    for k in range(n):
        a[k, k] = math.sqrt(a[k, k] - np.dot(a[k, 0:k], a[k, 0:k]))

        for i in range(k + 1, n):
            a[i, k] = (a[i, k] - np.dot(a[i, 0:k], a[k, 0:k])) / a[k, k]

    for k in range(1, n):
        a[0:k, k] = 0.

    return a


def choleski_solve(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    n = len(b)

    for k in range(n):
        b[k] = (b[k] - np.dot(a[k, 0:k], b[0:k])) / a[k, k]

    for k in range(n - 1, -1, -1):
        b[k] = (b[k] - np.dot(a[k + 1:n, k], b[k + 1:n])) / a[k, k]

    return b


def lu_3_diag_decomposition(c: np.ndarray, d: np.ndarray, e: np.ndarray):
    n = len(d)

    for k in range(1, n):
        l = c[k - 1] / d[k - 1]
        d[k] = d[k] - l * e[k - 1]
        c[k - 1] = l

    return c, d, e


def lu_3_diag_solve(c: np.ndarray, d: np.ndarray, e: np.ndarray, b: np.ndarray):
    n = len(d)

    for k in range(1, n):
        b[k] = b[k] - c[k - 1] * b[k - 1]

    b[n - 1] = b[n - 1] / d[n - 1]

    for k in range(n - 2, -1, -1):
        b[k] = (b[k] - e[k] * b[k + 1]) / d[k]

    return b


def matrix_3_diag_mult_vector(
    c: np.ndarray, d: np.ndarray, e: np.ndarray, 
    x: np.ndarray
) -> np.ndarray:
    n = len(d)
    y = np.zeros(n)

    for i in range(n):
        ce = c[i - 1] if i > 0 else 0
        xc = x[i - 1] if i > 0 else 0

        de = d[i]
        xd = x[i]

        ee = e[i] if i < n - 1 else 0
        xe = x[i + 1] if i < n - 1 else 0

        y[i] = ce * xc + de * xd + ee * xe

    return y


In [34]:
a = np.array([
    [13., -4., 1.],
    [-4., 13., -4.],
    [1.,  -4., 13.]
])

b = np.array([-112., 661., -113.])

In [35]:
x = gauss_elimination(np.copy(a), np.copy(b))

print("Метод Гаусса")
print(f"Решение {x}")
print(f"Ошибка {np.dot(a, x) - b}")

Метод Гаусса
Решение [ 7.91833333 55.69333333  7.835     ]
Ошибка [-1.42108547e-14  0.00000000e+00 -1.42108547e-14]


In [36]:
lu = lu_decomposition(a.copy())

x = lu_solve(lu, b.copy())

print("LU разложение")
print(f"Решение {x}")
print(f"Ошибка {np.dot(a, x) - b}")

LU разложение
Решение [ 7.91833333 55.69333333  7.835     ]
Ошибка [-1.42108547e-14  0.00000000e+00 -1.42108547e-14]


In [37]:
chol = choleski_decomposition(a.copy())
x = choleski_solve(chol, b.copy())

print("Метод Холецкого")
print(f"Решение {x}")
print(f"Ошибка {np.dot(a, x) - b}")

Метод Холецкого
Решение [ 7.91833333 55.69333333  7.835     ]
Ошибка [ 0.00000000e+00  0.00000000e+00 -1.42108547e-14]


In [38]:


a = np.array([
    [19., -4., 0., 0., 0.],
    [-4., 19., -4., 0, 0.],
    [0.,  -4., 19., -4, 0],
    [0., 0., -4., 19., -4],
    [0., 0., 0., -4., 19.]
])
d = np.ones((5)) * 5.
c = np.ones((4)) * (-1)

b = np.array([1115., 6664., 3333., 6662., 5551.])

e = c.copy()

ct, dt, et = lu_3_diag_decomposition(c.copy(), d.copy(), e.copy())
x = lu_3_diag_solve(ct, dt, et, b.copy())


print("LU для трехдиагональной матрицы")
print(f"Решение {x}")
print(f"Ошибка {matrix_3_diag_mult_vector(c, d, e, x) - b}")

LU для трехдиагональной матрицы
Решение [ 568.0219697  1725.10984848 1393.52727273 1909.52651515 1492.10530303]
Ошибка [2.27373675e-13 1.81898940e-12 0.00000000e+00 0.00000000e+00
 0.00000000e+00]
